In [0]:
%sql
SELECT column_name, data_type
FROM nyc_taxi.information_schema.columns
WHERE table_name = 'bronze_data'
  AND table_schema = 'default';

column_name,data_type
VendorID,INT
tpep_pickup_datetime,TIMESTAMP
tpep_dropoff_datetime,TIMESTAMP
passenger_count,INT
trip_distance,DOUBLE
pickup_longitude,DOUBLE
pickup_latitude,DOUBLE
RateCodeID,INT
store_and_fwd_flag,STRING
dropoff_longitude,DOUBLE


### 1. Find the total number of trips in Jan 2015 and compare with Jan–Mar 2016.

In [0]:
%sql
select count(*) as number_of_trips,year(tpep_pickup_datetime) as year,month(tpep_pickup_datetime) as month from nyc_taxi.default.silver_data group by year,month order by year,month

number_of_trips,year,month
12748986,2015,1
10906858,2016,1
11382049,2016,2
12210952,2016,3


### 2. Calculate the average trip distance and average fare amount.

In [0]:
%sql
select concat(round(avg(trip_distance),2)," miles") as average_trip_distance,concat("$",round(avg(fare_amount),2)) as average_fare_amount from nyc_taxi.default.silver_data;

average_trip_distance,average_fare_amount
7.51 miles,$12.39


### 3.Find the busiest pickup hour of the day.

In [0]:
%sql
select hour(tpep_pickup_datetime) as Busiest_hour,count(*) as number_of_trips from nyc_taxi.default.silver_data group by Busiest_hour order by number_of_trips desc limit 1

number_of_trips,Busiest_hour
2986286,18


### 4. Calculate the total revenue per month

In [0]:
%sql
select year(tpep_pickup_datetime) as year,month(tpep_pickup_datetime) as month,sum(total_amount) as total_revenue from nyc_taxi.default.silver_data group by year,month order by year,month

year,month,total_revenue
2015,1,1.9261543553201282E8
2016,1,1.7059847688128346E8
2016,2,1.7758995918046305E8
2016,3,1.9593473383996147E8


### 5. Find the top 5 days with highest total revenue.

In [0]:
%sql
select date(tpep_pickup_datetime) as date,round(sum(total_amount),1) as total_revenue from nyc_taxi.default.silver_data group by date order by total_revenue desc limit 5

date,total_revenue
2015-01-18,1.00009116E7
2015-01-30,7351182.7
2016-02-12,7272147.6
2015-01-31,7271657.4
2015-01-16,7252079.5


### 6. Top 10 most common pick ups

In [0]:
%sql
select pickup_latitude,pickup_longitude,count(*) as total_trips from nyc_taxi.default.silver_data group by pickup_latitude,pickup_longitude order by total_trips desc limit 10

pickup_latitude,pickup_longitude,total_trips
0.0,0.0,772525
40.82100296020508,-73.95466613769531,1523
40.74489974975586,-73.94863891601562,1043
40.69314193725586,-74.1863021850586,729
40.76948547363281,-73.86331939697266,520
40.7222900390625,-73.9867172241211,431
40.74357604980469,-73.91512298583984,307
40.774051666259766,-73.87445831298828,301
40.773738861083984,-73.8708724975586,299
40.64537048339844,-73.77670288085938,285


### 7. Top 10 most common drops

In [0]:
%sql
select dropoff_latitude,dropoff_longitude,count(*) as total_trips from nyc_taxi.default.silver_data group by dropoff_latitude,dropoff_longitude order by total_trips desc limit 10

dropoff_latitude,dropoff_longitude,total_trips
0.0,0.0,733832
40.82100296020508,-73.95466613769531,1523
40.74489974975586,-73.94863891601562,1043
40.69314193725586,-74.1863021850586,729
40.76948547363281,-73.86331939697266,503
40.7222900390625,-73.9867172241211,429
40.74357604980469,-73.91512298583984,322
40.72767639160156,-74.00314331054688,233
40.731502532958984,-73.98845672607422,158
40.691463470458984,-73.92151641845703,153


### 8. Find the longest trip (by distance) and highest fare trip.

In [0]:
%sql
-- Longest Trip Details
SELECT *
FROM nyc_taxi.default.silver_data
WHERE trip_distance = (
    SELECT MAX(trip_distance) FROM nyc_taxi.default.silver_data
);

-- Highest Fare Trip Details
SELECT *
FROM nyc_taxi.default.silver_data
WHERE fare_amount = (
    SELECT MAX(fare_amount) FROM nyc_taxi.default.silver_data
);


VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,tip_amount,total_amount
1,2016-03-10T22:59:51.000Z,2016-03-11T00:18:58.000Z,1,0.0,-73.99555206298828,40.71595001220703,-73.9388656616211,40.82597351074219,2,429496.72,0.0,429562.25


### 9. Calculate the percentage of trips with passenger_count > 3.

In [0]:
%sql
select
round(
(count(case when passenger_count>3 then 1 end) * 100.0) / count(*) ,2) as percent
from nyc_taxi.default.silver_data

percent
10.73


### 10. Identify if extra charges (MTA_tax, surcharge) significantly affect final fare.

In [0]:
%sql
select case when (extra+mta_tax+improvement_surcharge)>0 then "Extra" else "No extra" end as affecting,count(*) as total_trips,sum(total_amount) as amount from nyc_taxi.default.bronze_data group by affecting;

extra_flag,trip_count,avg_total_amount
With_Extras,47221988,15.6
No_Extras,26857,8.17
